In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
os.environ['PATH']+= r'C:/SeleniumDrivers'

In [119]:
driver=webdriver.Chrome()
driver.get('https://world.openfoodfacts.org/country/india')

In [108]:
elements=driver.find_elements(By.CSS_SELECTOR,'div.list_product_name')

In [109]:
for i in elements:
    print(i.text)

Red bull energy drink - 250 ml
Sprite - coca cola - 750 ml
Heinz Tomato ketchup - 400ml
Thumbs Up - Coca Cola Company - 750ml
Sting - 250 ml
kinley mineral water - 1 l
Sprite - 2.25l
Mountain Dew - Pepsico
maaza - Hisdustan Unilever Limited - 600 ml
Kinley Mineral Water - 1 L
Limca - Coca-Cola
Thums Up (Coca-Cola)
HERSHEY’S CHOCOLATE SYRUP - 623 g
Cadbury gems - 7.9g
Taka tak - Haldiram's
Thums up - 250 ml
Chyawanprash - Dabur - 550 g
Good Day Butter Cookies - Britannia - 68g
Sugar cookie - CMB - 100 g
Frooti - 2 l
Diet coke - The Coca-Cola Company - 500 ml
Mountain dew - mountaindew - 2.25
7up - PepsiCo - 750 ml
Coca Cola - 750 ml
Rooh Afza - hamdard - 750 ml
Marie Gold - Britannia - 73 g
Pepsi - 1,5 L
Pepsi 2.25 l - Pepsico
Parle - 22g
Bourn vita - 500g
Eno, fruit salt lemon flavour - GSK - 5g
Maaza Original - - 1.2 l
coco cola company - sprite - 250 ml
Barritas de platano y chocolate - Alteza
Oreo
Frooti mango drink - parle Agro - 150ml
thumsup - recycle
ORSL Plus Electrolyte Drink 

In [120]:
product_link = driver.find_elements(By.CSS_SELECTOR, 'a.list_product_a')

In [121]:
product_linkies=[]
for i in product_link:
    product_linkies.append(i.get_attribute('href'))

In [122]:
def individual():
    try:
        prod_name = driver.find_element(By.CSS_SELECTOR, 'h2.title-1').text.strip()
        rows = driver.find_elements(By.XPATH, "//tbody/tr")
        if not prod_name or not rows:
            return pd.DataFrame()
        nutrition_data = {"Product Name": [prod_name]}
        for row in rows:
            nutrient = row.find_element(By.XPATH, "./td[1]").text.strip()
            as_sold_100g = row.find_element(By.XPATH, "./td[2]").text.strip()
            nutrition_data[nutrient] = [as_sold_100g]
        df = pd.DataFrame(nutrition_data)
        return df
    except NoSuchElementException:
        return pd.DataFrame()

In [123]:
for product_link in product_linkies:
    main_window = driver.current_window_handle
    driver.execute_script("window.open();")
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(product_link)
    try:
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,"//tbody/tr")))
        product_df=individual()
        master_df=pd.concat([master_df, product_df], ignore_index=True)
    except TimeoutException:
        print(f"Timeout: Table not found for {product_link}")
    
    driver.close()
    driver.switch_to.window(main_window)

print(master_df)


Timeout: Table not found for https://world.openfoodfacts.org/product/8902080364053/mountain-dew-pepsico
                              Product Name                Energy     Fat  \
0           Red bull energy drink - 250 ml     194 kj\n(46 kcal)     0 g   
1              Sprite - coca cola - 750 ml     167 kj\n(40 kcal)     0 g   
2             Heinz Tomato ketchup - 400ml    435 kj\n(102 kcal)   0.1 g   
3    Thumbs Up - Coca Cola Company - 750ml     151 kj\n(36 kcal)     0 g   
4                           Sting - 250 ml                  -34%   -100%   
..                                     ...                   ...     ...   
116    Noix de cajou nature - Boni - 250 g  2,431 kj\n(581 kcal)    44 g   
117         Tamari soya sauce - Lima - 1 l     402 kj\n(96 kcal)     0 g   
118   Maggi Masala Noodles - Nestle - 140g  1,787 kj\n(427 kcal)  15.7 g   
119                          kurkure - 25g                   NaN       ?   
120     Cadbury Dairy Milk Chocolate - 24g  2,234 kj\n(534 k

In [124]:
master_df

,Product Name,Energy,Fat,Saturated fat,Trans fat,Cholesterol,Carbohydrates,Sugars,Fiber,Proteins,...,Vitamin A,Paper or cardboard,Vitamin E,Vitamin C (ascorbic acid),Chloride,Zinc,Copper,Selenium,Folates (total folates),Cocoa (minimum)
0,Red bull energy drink - 250 ml,194 kj\n(46 kcal),0 g,0 g,0 g,0 mg,11 g,11 g,0 g,0 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sprite - coca cola - 750 ml,167 kj\n(40 kcal),0 g,0 g,NaN,NaN,10 g,9.8 g,0 g,0 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Heinz Tomato ketchup - 400ml,435 kj\n(102 kcal),0.1 g,< 0.1 g,NaN,NaN,23.2 g,22.8 g,?,1.2 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Thumbs Up - Coca Cola Company - 750ml,151 kj\n(36 kcal),0 g,0 g,NaN,NaN,9 g,9 g,?,0 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sting - 250 ml,-34%,-100%,-100%,NaN,NaN,-30%,-27%,,-100%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Noix de cajou nature - Boni - 250 g,"2,431 kj\n(581 kcal)",44 g,7.8 g,NaN,NaN,27 g,5.9 g,3.3 g,18 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,Tamari soya sauce - Lima - 1 l,402 kj\n(96 kcal),0 g,0 g,NaN,NaN,5.3 g,1.7 g,0.9 g,12 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,Maggi Masala Noodles - Nestle - 140g,"1,787 kj\n(427 kcal)",15.7 g,6.8 g,NaN,NaN,63.5 g,1.2 g,3.6 g,8 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,kurkure - 25g,NaN,?,?,NaN,NaN,?,?,?,?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
master_df.to_excel('output.xlsx', index=False)